In [1]:
import numpy as np
import pandas as pd
import os
import PIL
from PIL import Image
import time
from glob import glob
import matplotlib.pyplot as plt

In [2]:
import torch
from torch import nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

## Datasets dan Dataloader

In [3]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [4]:
# function to find location from training data and test data 

def Path(location) :
    root = os.path.join('data', '*')
    root = glob(root)
    
    if location == "train_sets" :
        path = root[1]
    
    elif location == "test_sets" :
        path = root[0]
    
    return path

In [5]:
bs = 32
crop_size = 128

#pipeline data augmentation 
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(crop_size, scale=(0.8, 1.0)),
    transforms.RandomVerticalFlip(),
    transforms.RandomHorizontalFlip(),
    transforms.RandomAffine(0, shear=(10)),
    transforms.RandomRotation(20),
    transforms.ToTensor()
])

test_transform = transforms.Compose([
    transforms.Resize(135),
    transforms.CenterCrop(crop_size),
    transforms.ToTensor()
])

In [6]:
train_set = datasets.ImageFolder(Path('train_sets'), transform=train_transform)
test_set = datasets.ImageFolder(Path('test_sets'), transform=test_transform)

In [7]:
print (train_set)
print (test_set)

Dataset ImageFolder
    Number of datapoints: 596
    Root location: data/train_sets
    StandardTransform
Transform: Compose(
               RandomResizedCrop(size=(128, 128), scale=(0.8, 1.0), ratio=(0.75, 1.3333), interpolation=bilinear)
               RandomVerticalFlip(p=0.5)
               RandomHorizontalFlip(p=0.5)
               RandomAffine(degrees=[0.0, 0.0], shear=[-10.0, 10.0])
               RandomRotation(degrees=[-20.0, 20.0], interpolation=nearest, expand=False, fill=0)
               ToTensor()
           )
Dataset ImageFolder
    Number of datapoints: 252
    Root location: data/test_train
    StandardTransform
Transform: Compose(
               Resize(size=135, interpolation=bilinear, max_size=None, antialias=None)
               CenterCrop(size=(128, 128))
               ToTensor()
           )


In [8]:
# train loader, valid_loader, test_loader
train_loader = DataLoader(train_set, batch_size=bs, shuffle=True, num_workers=4)
test_loader = DataLoader(test_set, batch_size=bs, shuffle=True)

In [9]:
feature, target = next(iter(train_loader))
feature.shape

KeyboardInterrupt: 

In [ ]:
label2cat = train_set.classes
label2cat

## model architecture

In [ ]:
class My_model(nn.Module) :
    def __init__(self) :
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=8, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Flatten()
        )
        self.fc = nn.Sequential(
            nn.Linear(in_features=2048, out_features=1024),
            nn.ReLU(),
            nn.Linear(in_features=1024, out_features=7),
            nn.LogSoftmax(1)
        )
    def forward(self, x) :
        x = self.conv(x)
        x = self.fc(x)
        return x
        

## Training preperation

In [ ]:
from torch import optim
from tqdm.auto import tqdm

In [ ]:
lr = 0.001

model = My_model().to(device)
criterion = nn.NLLLoss()
optimizer = optim.AdamW(model.parameters(), lr= lr)

In [ ]:
#function looping
def looping(mode, dataset, dataloader, model, criterion, optimizer, device) :
    if mode =="train" :
        model.train()
    
    elif mode =="test" :
        model.eval()
    
    cost = correct = 0
    for feature, target in tqdm(dataloader, desc=mode.title()) :
        feature, target = feature.to(device), target.to(device)
        output = model(feature)
        loss = criterion(output, target)
        
        if mode =="train" :
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        
        cost += loss.item() * feature.shape[0]
        correct += (output.argmax(1) == target).sum().item()
        
    cost = cost / len(dataset)
    acc = correct / len(dataset)
    
    return cost, acc

In [ ]:
#test

epochs = 12
train_cost, test_cost = [], []
train_acc, test_acc = [], []
for i in range (epochs) :
    since = time.time()
    # training for data train
    cost, acc = looping("train", train_set, train_loader, model, criterion, optimizer, device)
    train_cost.append(cost)
    train_acc.append(acc)
    
    # training for data val
    with torch.no_grad() :
        cost, acc = looping("test", test_set, test_loader, model, criterion, optimizer, device)
        test_cost.append(cost)
        test_acc.append(acc)
    
    
    print(f"Epoch: {i+1}/{epochs} | train_cost : {train_cost[-1]} | test_cost : {test_cost[-1]} | "
         f"train_acc: {train_acc[-1]} | test_acc : {test_acc[-1]}")